# ARIMA / GARCH Based Recovery Recommendations
___

### TOC

- [Intro](#hrv)
    - [HRV](#hrv)
    - [ARIMA and GARCH](#arima-garch)
    - [Possible Applications](#examples)
- [Methodology](#methodology)
- [Assumption of]:
    - Stationarity
    - Seasonality
    
    
### Intro – HRV<a id="hrv">
[Heart Rate Variability](https://en.wikipedia.org/wiki/Heart_rate_variability) is the primary meaure of preparedness for stress which My Oura ring uses to advise me on how much exercise would be ideal for me for that day. It lines up quite well with my subjective feeling, but an intra-day comparison shows that there's zero correlation. Of course, I really ought to consider the few days of data before. I'll get around to that.

### Intro – ARIMA GARCH <a id="arima-garch">
In any case, I thought i'd try throwing [ARIMA](https://python-bloggers.com/2021/04/arima-model-in-python/) and GARCH at my own health data and seeing what would come of it. The ARIMA model can be checked against future data to see if it can make predictions on future data. On the GARCH side of things, I can turn it into a product. Something that makes predictions dynamically. Originally, ARCH/GARCH is a model designed for making econometric predictions by taking into consideration autocorrelative implications for hetroskedasticity - in english, (G)ARCH is a prediction model that is backwards-looking (in time, when applied to timeseries) and takes into consideration the volatility of previous data.

### Examples <a id="examples">

Application wise, there's a few immediate ideas that come to mind for an integrated sytem:

Example 1
> I'm still out past my bed time? It automatically guesses how much sleep I need and based on that expectation, adjusts my Hue light auto-on feature and my alarm to match it appropriately. Of course, I can sign off on the suggestion once I'm in bed or make changes.

Exmaple 2
> 

### Methodology <a id="methodology">

1. Use


    
### biblio

[How to ARIMA](https://python-bloggers.com/2021/04/arima-model-in-python/)

In [2]:
import numpy as np
import pandas as pd

In [13]:
# --- Get Oura Data

df = pd.read_csv("data/oura_clean.csv")
df.drop("Unnamed: 0", axis=1, inplace=True)
df.set_index('date', inplace=True)
display(df.head())

,sleep_score,total_sleep_score,rem_sleep_score,deep_sleep_score,sleep_efficiency_score,restfulness_score,sleep_latency_score,sleep_timing_score,total_bedtime,total_sleep_time,...,long_periods_of_inactivity,readiness_score,previous_night_score,sleep_balance_score,previous_day_activity_score,activity_balance_score,temperature_score,resting_heart_rate_score,hrv_balance_score,recovery_index_score
date,,,,,,,,,,,,,,,,,,,,,
2020-01-01,75.0,71.0,61.0,95.0,74.0,52.0,91.0,100.0,31380.0,25440.0,...,0,75.0,60.0,66.0,81.0,89.0,100.0,95.0,NaN,69.0
2020-01-02,75.0,71.0,52.0,95.0,72.0,56.0,97.0,100.0,31680.0,25200.0,...,0,73.0,60.0,66.0,100.0,88.0,100.0,53.0,NaN,100.0
2020-01-03,67.0,62.0,9.0,93.0,72.0,54.0,94.0,100.0,28980.0,23010.0,...,0,72.0,40.0,64.0,98.0,88.0,95.0,100.0,NaN,58.0
2020-01-04,71.0,63.0,62.0,66.0,79.0,53.0,99.0,100.0,28140.0,23280.0,...,0,74.0,50.0,61.0,97.0,92.0,92.0,79.0,NaN,95.0
2020-01-05,64.0,64.0,27.0,99.0,65.0,62.0,64.0,67.0,30300.0,23460.0,...,0,60.0,33.0,60.0,91.0,86.0,53.0,54.0,NaN,91.0


In [ ]:
# --- Select out variables of interest

variables_of_interst = [
    ""
]

### 1. Coordinate Assumptions

First determine whether or not the data is stationary

In [ ]:
# --- Check for Stationarity

from statsmodels.tsa.stattools import adfuller

